## UTH-BERT

本ノートブックは、下記ページを参考に作成
https://qiita.com/Kunikata/items/d9fda2351a273a7412f6

形態素解析用の辞書をインストールする部分、データセットをダウンロードする部分は省略（上記ページを参照のこと）

### pythonライブラリのインポート

In [1]:
import os, sys
import tensorflow as tf
import pandas as pd
import numpy as np
import keras_bert

### データセットとモデルの読み込み

In [2]:
# データセット
knbc_dir_path = 'KNBC_v1.0_090925_utf8'
gourmet_tsv_file_path = os.path.join(knbc_dir_path, 'corpus2/Gourmet.tsv')
keitai_tsv_file_path = os.path.join(knbc_dir_path, 'corpus2/Keitai.tsv')
kyoto_tsv_file_path = os.path.join(knbc_dir_path, 'corpus2/Kyoto.tsv')
sports_tsv_file_path = os.path.join(knbc_dir_path, 'corpus2/Sports.tsv')

# 訓練済みモデル
pretrained_model_dir_path = 'UTH_BERT_BASE_MC_BPE_V25000_10M'
pretrained_bert_config_file_path = os.path.join(pretrained_model_dir_path, 'bert_config.json')
pretrained_model_checkpoint_path = os.path.join(pretrained_model_dir_path, 'model.ckpt-10000000') # 拡張子不要
pretrained_vocab_file_path = os.path.join(pretrained_model_dir_path, 'vocab.txt')

# 今回学習するモデル
!mkdir train_model
train_model_dir_path = 'train_model'
train_bert_config_file_path = os.path.join(train_model_dir_path, 'train_bert_config.json')
train_model_checkpoint_path = os.path.join(train_model_dir_path, 'train_model.ckpt')

mkdir: ディレクトリ `train_model' を作成できません: ファイルが存在します


### データセットの作成

In [3]:
# 各カテゴリのtsvファイルを読み込んでラベル列を追加
df_gourmet = pd.read_table(gourmet_tsv_file_path, header=None)
df_gourmet['label'] = 'グルメ'

df_keitai = pd.read_table(keitai_tsv_file_path, header=None)
df_keitai['label'] = '携帯電話'

df_kyoto = pd.read_table(kyoto_tsv_file_path, header=None)
df_kyoto['label'] = '京都観光'

df_sports = pd.read_table(sports_tsv_file_path, header=None)
df_sports['label'] = 'スポーツ'

# 結合して必要な列だけ抽出
df_dataset = pd.concat([df_gourmet, df_keitai, df_kyoto, df_sports])[[1, 'label']]
df_dataset.columns = ['text', 'label'] # ラベル名変更
df_dataset

,text,label
0,［グルメ］烏丸六角のおかき屋さん,グルメ
1,六角堂の前にある、蕪村庵というお店に行ってきた。,グルメ
2,おかきやせんべいの店なのだが、これがオイシイ。,グルメ
3,のれんをくぐると小さな庭があり、その先に町屋風の店内がある。,グルメ
4,せんべいの箱はデパートみたいな山積みではなく、間隔をあけて陳列されているのがまた良い。,グルメ
...,...,...
517,筋力が違う！！,スポーツ
518,なんか神様、不公平・・・,スポーツ
519,男性諸君、このこと忘れないでやぁ（＞◆＜）,スポーツ
520,まぁ。。。,スポーツ


### 学習・テストデータに分割

In [4]:
from sklearn.model_selection import train_test_split
df_train, df_test = train_test_split(df_dataset, test_size=500)

In [5]:
# sed コマンドで tf.gfile.GFile を tf.io.gfile.GFile に置換
!sed -i-e 's/tf\.gfile\.GFile/tf\.io\.gfile\.GFile/g' ./uth_bert/tokenization_mod.py

In [6]:
### テキストの前処理

In [7]:
from uth_bert.preprocess_text import preprocess as my_preprocess
from uth_bert.tokenization_mod import MecabTokenizer, FullTokenizerForMecab

### 形態素解析のテスト

公式（https://github.com/jinseikenai/uth-bert の example_main.py）の内容が実行できることを確認します。

In [8]:
# NEOlogdのパス
neologd_dic_dir_path = '/usr/lib/x86_64-linux-gnu/mecab/dic/mecab-ipadic-neologd'

# 万病辞書のパス
manbyo_dic_path = './MANBYO_201907_Dic-utf8.dic'

# special token for a Person's name (Do not change)
name_token = "＠＠Ｎ"

# path to the mecab-ipadic-neologd
#mecab_ipadic_neologd = '/usr/lib/mecab/dic/mecab-ipadic-neologd' # 変更
mecab_ipadic_neologd = neologd_dic_dir_path

# path to the J-Medic (We used MANBYO_201907_Dic-utf8.dic)
#mecab_J_medic = './MANBYO_201907_Dic-utf8.dic' # 変更
mecab_J_medic = manbyo_dic_path

# path to the uth-bert vocabulary
#vocab_file = "./bert_vocab_mc_v1_25000.txt" # 変更
vocab_file = pretrained_vocab_file_path

# MecabTokenizer
sub_tokenizer = MecabTokenizer(mecab_ipadic_neologd=mecab_ipadic_neologd,
                                mecab_J_medic=mecab_J_medic,
                                name_token=name_token)

# FullTokenizerForMecab
tokenizer = FullTokenizerForMecab(sub_tokenizer=sub_tokenizer,
                                    vocab_file=vocab_file,
                                    do_lower_case=False)

In [9]:
# pre-process and tokenize example
original_text = "2002 年夏より重い物の持ち上げが困難になり，階段の昇りが遅くなるなど四肢の筋力低下が緩徐に進行した．2005 年 2 月頃より鼻声となりろれつが回りにくくなった．また，食事中にむせるようになり，同年 12 月に当院に精査入院した。"
print ('元のテキスト\n', original_text, end='\n\n')

pre_processed_text = my_preprocess(original_text)
print ('前処理後テキスト\n', pre_processed_text, end='\n\n')

output_tokens = tokenizer.tokenize(pre_processed_text)
print ('トークン化後のテキスト\n', output_tokens)

元のテキスト
 2002 年夏より重い物の持ち上げが困難になり，階段の昇りが遅くなるなど四肢の筋力低下が緩徐に進行した．2005 年 2 月頃より鼻声となりろれつが回りにくくなった．また，食事中にむせるようになり，同年 12 月に当院に精査入院した。

前処理後テキスト
 ２００２年夏より重い物の持ち上げが困難になり、階段の昇りが遅くなるなど四肢の筋力低下が緩徐に進行した．２００５年２月頃より鼻声となりろれつが回りにくくなった．また、食事中にむせるようになり、同年１２月に当院に精査入院した。

トークン化後のテキスト
 ['２００２年', '夏', 'より', '重い', '物', 'の', '持ち上げ', 'が', '困難', 'に', 'なり', '、', '階段', 'の', '[UNK]', 'が', '遅く', 'なる', 'など', '四肢', 'の', '筋力低下', 'が', '緩徐', 'に', '進行', 'し', 'た', '．', '２００５年', '２', '月頃', 'より', '鼻', '##声', 'と', 'なり', 'ろ', '##れ', '##つ', 'が', '回り', '##にく', '##く', 'なっ', 'た', '．', 'また', '、', '食事', '中', 'に', 'むせる', 'よう', 'に', 'なり', '、', '同年', '１２月', 'に', '当', '院', 'に', '精査', '入院', 'し', 'た', '。']


同じになることを確認。「##」がついているのはサブワードを表している。

In [10]:
def preprocess_text(s):
    result = []
    for text in s:
        result.append(my_preprocess(text))

    return result

train_text_preprocessed = preprocess_text(df_train['text'])
test_text_preprocessed = preprocess_text(df_test['text'])

# 先頭3つのデータを表示
for i in range(3):
    print(train_text_preprocessed[i])

これが純粋に損。
ａｕ／ツーカー
こうしたことを避けたければ、返信機能を使わずに意見を募集した人のメアドを電話帳などで調べて、普通にメールを送ればよかったのである。


文頭に分類問題を表す[CLS]を、文末に文章の終わりであることを表す[SEP]を挿入する。

In [11]:
def tokenize_text(s):
    result = []
    for text in s:
        result.append(['[CLS]'] + tokenizer.tokenize(text) + ['[SEP]'])

    return result

train_text_tokenized = tokenize_text(df_train['text'])
test_text_tokenized = tokenize_text(df_test['text'])

# 先頭3つのデータを表示
for i in range(3):
    print(train_text_tokenized[i])

['[CLS]', 'これ', 'が', '[UNK]', 'に', '損', '。', '[SEP]']
['[CLS]', 'ａｕ', '／', 'ツ', '##ー', '##カー', '[SEP]']
['[CLS]', 'こう', '##した', 'こと', 'を', '避け', 'た', '##けれ', 'ば', '、', '返信', '機能', 'を', '使わ', 'ず', 'に', '意見', 'を', '募', '##集', 'し', 'た', '人', 'の', 'メ', '##アド', 'を', '電話', '##帳', 'など', 'で', '調べ', 'て', '、', '普通', '##に', 'メール', 'を', '送れ', 'ば', 'よかっ', 'た', 'の', 'で', 'ある', '。', '[SEP]']


今回のデータセットにおける文章の最大長を獲得する。

In [12]:
# ファインチューニングする場合は学習データ・テストデータの最大長を用いる
maxlen = 0

for tokens in train_text_tokenized:
    maxlen = max(maxlen, len(tokens))

for tokens in test_text_tokenized:
    maxlen = max(maxlen, len(tokens))

maxlen

140

トークンをidに変換する。

In [13]:
def tokens_to_ids(tokenized_text):
    result = []
    for tokens in tokenized_text:
        result.append(tokenizer.convert_tokens_to_ids(tokens))

    return result

train_text_ids = tokens_to_ids(train_text_tokenized)
test_text_ids = tokens_to_ids(test_text_tokenized)

# 先頭3つのデータを表示
for i in range(3):
    print(train_text_ids[i])

[2, 539, 18, 1, 14, 17658, 6, 3]
[2, 12509, 11, 3933, 2023, 9722, 3]
[2, 3736, 9124, 48, 25, 3446, 19, 23038, 142, 5, 5154, 437, 25, 4666, 100, 14, 3569, 25, 24114, 6572, 16, 19, 693, 10, 8997, 9027, 25, 956, 22073, 133, 21, 5500, 13, 5, 1011, 3046, 6685, 25, 19283, 142, 2017, 19, 10, 21, 58, 6, 3]


pandasデータセットをnumpy形式に変換。文長が140に満たない場合は、後ろに0を挿入する。

In [14]:
def list_to_numpy_array(input_list, maxlen):
    result = np.zeros((len(input_list), maxlen), dtype=np.int32)

    for i in range(len(input_list)):
        for j in range(len(input_list[i])):
            result[i][j] = input_list[i][j]

    return result

X_train = list_to_numpy_array(train_text_ids, maxlen)
X_test = list_to_numpy_array(test_text_ids, maxlen)

# 先頭3つのデータを表示
X_train[:3]

array([[    2,   539,    18,     1,    14, 17658,     6,     3,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
      

正解ラベルの作成（One-Hot表現にする）

In [15]:
# ラベル -> インデックス の対応
label2index = {k: i for i, k in enumerate(df_train['label'].unique())}

# インデックス -> ラベル の対応
index2label = {i: k for i, k in enumerate(df_train['label'].unique())}

# ラベルの分類クラス数
class_count = len(label2index)
print('class count = ', class_count)

# One-hot encoding
y_train = tf.keras.utils.to_categorical([label2index[label] for label in df_train['label']], num_classes=class_count)
y_test = tf.keras.utils.to_categorical([label2index[label] for label in df_test['label']], num_classes=class_count)

# 先頭3つのデータを表示
print(y_train[:3])

class count =  4
[[1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]]


### 学習

学習済bertの設定ファイルを読み込む

In [16]:
import json
from pprint import pprint as pp

json_open = open(pretrained_bert_config_file_path, 'r')
pretrained_bert_config =json.load(json_open)

pp(pretrained_bert_config)

{'attention_probs_dropout_prob': 0.1,
 'hidden_act': 'gelu',
 'hidden_dropout_prob': 0.1,
 'hidden_size': 768,
 'initializer_range': 0.02,
 'intermediate_size': 3072,
 'max_position_embeddings': 512,
 'num_attention_heads': 12,
 'num_hidden_layers': 12,
 'type_vocab_size': 2,
 'vocab_size': 25000}


今回のデータセットに合わせ、学習済モデルの設定を変更する。

In [17]:
train_bert_config = pretrained_bert_config
train_bert_config['max_position_embeddings'] = maxlen
train_bert_config['max_seq_length'] = maxlen

pp(train_bert_config)

# jsonファイルとして保存
with open(train_bert_config_file_path, 'w') as f:
    json.dump(train_bert_config, f, indent=4)

{'attention_probs_dropout_prob': 0.1,
 'hidden_act': 'gelu',
 'hidden_dropout_prob': 0.1,
 'hidden_size': 768,
 'initializer_range': 0.02,
 'intermediate_size': 3072,
 'max_position_embeddings': 140,
 'max_seq_length': 140,
 'num_attention_heads': 12,
 'num_hidden_layers': 12,
 'type_vocab_size': 2,
 'vocab_size': 25000}


In [18]:
BATCH_SIZE = 4
EPOCHS = 1
LR = 1e-4

事前学習したときのパラメータをモデルに設定する。

In [19]:
from keras_bert import load_trained_model_from_checkpoint
bert = load_trained_model_from_checkpoint(train_bert_config_file_path, pretrained_model_checkpoint_path, training=True, trainable=True, seq_len=maxlen)
bert.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Input-Token (InputLayer)        [(None, 140)]        0                                            
__________________________________________________________________________________________________
Input-Segment (InputLayer)      [(None, 140)]        0                                            
__________________________________________________________________________________________________
Embedding-Token (TokenEmbedding [(None, 140, 768), ( 19200000    Input-Token[0][0]                
__________________________________________________________________________________________________
Embedding-Segment (Embedding)   (None, 140, 768)     1536        Input-Segment[0][0]              
_______________________________________________________________________________________

今回のタスクに合わせて、bert-encoderの最終層にClassification layerを追加する。

In [20]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout, LSTM, Bidirectional
from keras_bert import AdamWarmup, calc_train_steps

def _create_model(bert, maxlen, class_count):
    bert_last = bert.get_layer(name='NSP-Dense').output #  NSP-Denseを指定する理由は要確認
    output_tensor = Dense(class_count, activation='softmax')(bert_last)

    model = Model([bert.input[0], bert.input[1]], output_tensor)

    decay_steps, warmup_steps = calc_train_steps(
        maxlen,
        batch_size=BATCH_SIZE,
        epochs=EPOCHS,
    )

    # optimizer='nadam' では収束しないのでAdamWarmupを用いる
    model.compile(
        loss='categorical_crossentropy',
        optimizer=AdamWarmup(decay_steps=decay_steps, warmup_steps=warmup_steps, lr=LR),
        metrics=['acc', tf.keras.metrics.Precision(), tf.keras.metrics.Recall()]
    )

    return model

model = _create_model(bert, maxlen, class_count)
model.summary()

Model: "functional_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Input-Token (InputLayer)        [(None, 140)]        0                                            
__________________________________________________________________________________________________
Input-Segment (InputLayer)      [(None, 140)]        0                                            
__________________________________________________________________________________________________
Embedding-Token (TokenEmbedding [(None, 140, 768), ( 19200000    Input-Token[0][0]                
__________________________________________________________________________________________________
Embedding-Segment (Embedding)   (None, 140, 768)     1536        Input-Segment[0][0]              
_______________________________________________________________________________________

モデルの学習。

In [21]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

history = model.fit(
    [X_train, np.zeros_like(X_train)],
    y_train,
    epochs = EPOCHS,
    batch_size = BATCH_SIZE,
    validation_split=0.1,
    shuffle=True,
    verbose = 1,
    callbacks = [
        EarlyStopping(patience=5, monitor='val_acc', mode='max'),
        ModelCheckpoint(monitor='val_acc', mode='max', filepath=train_model_checkpoint_path, save_best_only=True)
    ]
)

830/830 [==============================] - ETA: 0s - loss: 1.3839 - acc: 0.3663 - precision: 0.4268 - recall: 0.0106WARNING:tensorflow:From /home/noro/anaconda3/envs/uthbert/lib/python3.7/site-packages/tensorflow/python/training/tracking/tracking.py:111: Model.state_updates (from tensorflow.python.keras.engine.training) is deprecated and will be removed in a future version.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: train_model/train_model.ckpt/assets
830/830 [==============================] - 416s 501ms/step - loss: 1.3839 - acc: 0.3663 - precision: 0.4268 - recall: 0.0106 - val_loss: 1.3825 - val_acc: 0.3767 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00


### テスト

In [22]:
from tensorflow.keras.models import load_model
from keras_bert import get_custom_objects

# 学習中に保存した最良モデルをロード (tensorflow2.1ではエラーになるので注意)
model = load_model(train_model_checkpoint_path, custom_objects=get_custom_objects())

# 予測を実行
y_test_pred_proba = model.predict([X_test, np.zeros_like(X_test)])

# 先頭3つのデータを表示
y_test_pred_proba[:3]

array([[0.19013445, 0.3285179 , 0.23311292, 0.24823469],
       [0.15091102, 0.34862372, 0.23078404, 0.26968125],
       [0.13056909, 0.36815464, 0.23028289, 0.27099344]], dtype=float32)

結果レポート

In [23]:
from sklearn.metrics import classification_report, confusion_matrix

y_test_true_labels = y_test.argmax(axis=1) # Probability -> index
y_test_pred_labels = y_test_pred_proba.argmax(axis=1) # One-hot -> index

target_names = [index2label[i] for i in range(class_count)]
rep = classification_report(y_test_true_labels, y_test_pred_labels, target_names=target_names, output_dict=True)

pd.DataFrame(rep)

/home/noro/anaconda3/envs/uthbert/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


,携帯電話,京都観光,スポーツ,グルメ,accuracy,macro avg,weighted avg
precision,0.500000,0.323293,0.0,0.0,0.324,0.205823,0.262100
recall,0.006329,1.000000,0.0,0.0,0.324,0.251582,0.324000
f1-score,0.012500,0.488619,0.0,0.0,0.324,0.125280,0.161285
support,158.000000,161.000000,70.0,111.0,0.324,500.000000,500.000000
